In [3]:
from chatbot.backend.services.vector_db.db import VectorDB
from chatbot.backend.services.models.models import vlm
from chatbot.backend.services.models.embedding_model import embedding_model
import requests

/opt/miniconda3/envs/odprt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
image_paths = ["archi.png", "34.png"] #"34.png", "43.png", "46.png", "160.png", 
image_paths = [f"chatbot/backend/sample_images/{path}" for path in image_paths]

In [4]:
print("Testing vlm")
image_summaries = vlm.generate_image_summaries(useful_image_paths=image_paths)

Testing vlm


In [5]:
print("Testing embedding model")
description_embeddings = embedding_model.batch_encode(image_summaries)

Testing embedding model


In [31]:
data = [
    {
        "doc_id": image_paths[i],
        "doc_source": "sample_images",
        "text": "",
        "text_dense_embedding": [0 for _ in range(1024)],  # Placeholder for text dense embedding
        "text_sparse_embedding": [],  # Empty sparse vector
        "description": image_summaries[i],
        "description_embedding": description_embeddings[i],  # Embedding for Image Descriptions
    }
    for i in range(len(image_paths))
]

In [38]:
payload = {
    'data' : data
}

In [39]:
url = "http://0.0.0.0:8000/vector-db/insert-documents/"

# Make the POST request to ingest the data
response = requests.post(url, json=payload)

# Check the response
if response.status_code == 200:
    print("Data ingested successfully")
else:
    print(f"Failed to ingest data: {response.status_code}, {response.text}")

Data ingested successfully


In [40]:
url = "http://0.0.0.0:8000/vector-db/image-search/"

In [41]:
data = {
    "query" : "Model Architecture"
}

In [43]:
# Test Hybrid Search
response = requests.post(url, json=data)
if response.status_code == 200:
    print("Data ingested successfully")
    print(response.json())
else:
    print(f"Failed to ingest data: {response.status_code}, {response.text}")

Data ingested successfully
{'context': 'Doc_id: chatbot/backend/sample_images/archi.png \n Description: The image outlines the architecture of a RAG (Retrieval-Augmented Generation) system. The RAG Workflow involves users interacting with a chatbot that queries a Milvus Vector Database for relevant information. The database ingests various document types (FAQs, emails, PPTX, PDFs, and Excel files) and retrieves top K similar chunks for the query. The chatbot stores user chat history in a Firebase Relational Database. The Deployment and Scaling section shows users interacting with a load balancer that routes queries to RAG servers. Analytics & Dashboard tracks user queries, common topics, and unanswered queries for human analysis and refinement. The legend categorizes the architecture components into generative steps, functions/APIs, databases, and servers.\n\nDoc_id: chatbot/backend/sample_images/archi.png \n Description: The image outlines the architecture of a RAG (Retrieval-Augmente

In [44]:
response.json()['context']

'Doc_id: chatbot/backend/sample_images/archi.png \n Description: The image outlines the architecture of a RAG (Retrieval-Augmented Generation) system. The RAG Workflow involves users interacting with a chatbot that queries a Milvus Vector Database for relevant information. The database ingests various document types (FAQs, emails, PPTX, PDFs, and Excel files) and retrieves top K similar chunks for the query. The chatbot stores user chat history in a Firebase Relational Database. The Deployment and Scaling section shows users interacting with a load balancer that routes queries to RAG servers. Analytics & Dashboard tracks user queries, common topics, and unanswered queries for human analysis and refinement. The legend categorizes the architecture components into generative steps, functions/APIs, databases, and servers.\n\nDoc_id: chatbot/backend/sample_images/archi.png \n Description: The image outlines the architecture of a RAG (Retrieval-Augmented Generation) system. The RAG Workflow 